# Introduction

## Summary

Today, we're going to be picking apart *Least Squares Model Selection*.
We'll be refining our theoretical argument from last lecture to guide us
here, investigating how well rankings of curves based on squared error
loss lines up with a rankings based on sample RMSE. 

We'll be using code from previous labs so we have some curves to rank
and choose from.

In [ ]:

suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
    CVXR::add_to_solver_blacklist('OSQP')  
})

## Styles

lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    	  			panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				  panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	              panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		          axis.ticks.x = element_blank(),
		          axis.ticks.y = element_blank(),
		          axis.text.x  = element_text(colour = "#aaaaaa"),
		          axis.text.y  = element_text(colour = "#aaaaaa"),
		          axis.title.x  = element_text(colour = "#aaaaaa"),
		          axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)

In [ ]:
## A function we'll want. unique with an inverse mapping.

invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

## Fitting Functions

monotonereg = function(X, Y, decreasing = FALSE) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list(if (decreasing) { diff(m) <= 0 } else { diff(m) >= 0 })

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "monotonereg"
  model
}

convexreg = function(X, Y, concave = FALSE, monotone = NULL) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  # Interpretation (rearrange): secant slopes are increasing
  uX = X[unique.X$elements]
  ii = 1:(n-2)
  constraints = 
    list(((m[ii+1]-m[ii])  * (uX[ii+2]-uX[ii]) - 
          (m[ii+2]-m[ii]) *  (uX[ii+1]-uX[ii])) * (-1)^concave <= 0)
  if(!is.null(monotone)) { 
       decreasing = monotone == 'decreasing'
       constraints = c(constraints, diff(m) * (-1)^decreasing >= 0)
  }
  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.convexreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "convexreg"
  model
}

bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( sum(abs(diff(m))) <= B )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "bvreg"
  model
}

lipreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  uX = X[unique.X$elements]
  constraints = list( abs(diff(m)) <= B * diff(uX) )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.lipreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "lipreg"
  model
}

## Prediction Functions

# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # i will be a vector of these numbers k, with one for each x in newdata$X
  i = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  Y[i]
}

predict.piecewise.linear = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X; n = length(X) 
  # for each new data point x[k]
  # find the closest observed X[i[k]] left of x[k]
  # i.e., i[k] is the largest integer i for which X[i] <= x[k] 
  i = findInterval(newdata$X, X) 
  # If there is no X[i] < x[k], findInterval sets i[k]=0
  #  and we'll want to act as if we'd gotten 1 so we use the
  #  line through (X[1], Y[1])  and (X[2], Y[2])
  # If that k is n, we'll want to act as if we'd gotten n-1 so we use 
  #  the line through (X[n-1], Y[n-1])  and (X[n], Y[n])
  i[i==0] = 1; i[i==n] = n-1
  # make a prediction using the formula y - y0 = (x-x0) * slope 
  Y[i] + (x-X[i]) * (Y[i+1]-Y[i])/(X[i+1]-X[i])
}

predict.monotonereg = predict.piecewise.constant
predict.convexreg = predict.piecewise.linear
predict.bvreg = predict.piecewise.constant
predict.lipreg = predict.piecewise.linear

## Conveniences

prediction.function = function(model) { 
  if(class(model)=='function') { model }
  else { function(x) { predict(model, newdata=data.frame(X=x)) } }
}


## Least Squares in Finite Models

Let's suppose we have independent and identically distributed
observations $(X_1,Y_1) \ldots (X_n,Y_n)$ where
$Y_i=\mu(X_i)+\varepsilon_i$ and $\varepsilon_i$ is normal with
mean-zero and standard deviation $\sigma$. We're going to be talking
about a least squares estimator.

$$ \hat\mu = \operatorname*{argmin}_{m \in \mathcal{M}} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 $$

Here's the error bound we proved in lecture. For a model $\mathcal{M}$
containing no more than $K$ curves,
$$ \lVert\hat\mu - \mu\rVert_{L_2(Pn)} \le 4\sigma\sqrt{\frac{\log(K)}{n}}  \ \text{ with probability } \ 1-1/K \ \textbf{ if } \ \mu \in \mathcal{M}. $$
This is not all that useful. It's a bound that's valid when we have a finite model (a set of finitely many curves) that contains $\mu$.
That's not going to happen.

The good news is that it's not a dead end. We can adapt the argument we
used to get a more general bound that has meaningful implications. 
$$ 
\lVert\hat\mu - \mu\rVert_{L_2(Pn)} \le 4\sigma\sqrt{\frac{\log(K)}{n}} + 3\lVert \mu^\star -\mu \rVert_{L_2(P_n)} \ \text{ with probability } \ 1-1/K \ \textbf{ for any } \ \mu^\star \in \mathcal{M}. 
$$
This bound applies whether or not $\mu$ is in our model $\mathcal{M}$. 
And what is says, if we take $\mu^star$ to be our model's best approximation
to $\mu$, that the curve we do select isn't much further than the best option
in our model. In the case that $\mu$ *is* in our model, that means taking $\mu^\star=\mu$, 
so this implies the bound above. There's a proof of this more general bound
at the end of this document. 

## Model selection

We're going to be talking about this in the context of a model selection problem.
The reason is that it's the only context anyone really
uses finite models. Consider these examples.

$$ 
\begin{aligned}
\mathcal{M}  &= \{m(x)=x^T b : b \in \mathbb{R}^k\} && \text{ Linear model } \\
\mathcal{M} &= \{ \text{ increasing } \ m(x) \} && \text{ Monotone model } \\
\mathcal{M}  &= \{ m(x) : \int_0^1 \lvert m'(x) \rvert dx \le B_{TV} \} && \text{ Bounded variation model } \\
\mathcal{M}  &= \{ m(x) : \max_{x \in [0,1]} \lvert m'(x) \rvert \le B_{Lip} \} && \text{ Lipschitz model }
\end{aligned}
$$

All of them contain infinitely many curves. But sometimes, we fit a few
models like this. After all, we don't know which is best. Maybe we're
choosing within a sort of *model family*, like these examples.

1.  Polynomial models of different orders.
2.  Models defined by different shape constraints, e.g. increasing vs.
    convex models.
3.  Balls of different radius in the same seminorm. For example,
    -   Bounded variation models for different bounds $B_{TV}$ on total
        variation.
    -   Lipschitz models for different bounds $B_{Lip}$ on the Lipschitz
        constant.

Or maybe we're choosing between as well as within families like this. In
any case, if we fit $K$ models $\mathcal{M}^1 \ldots \mathcal{M}^K$, we
get $K$ least squares estimators $\hat \mu^1 \ldots \hat \mu^K$. And we
can define a new, finite, model that contains these least squares
estimators. $$ \mathcal{M} = \{ \hat \mu^1 \ldots \hat \mu^K \}. $$
Choosing one of these curves, which if we prefer we can think of as
choosing a model from the set $\mathcal{M}^1 \ldots \mathcal{M}^K$, is the
**model selection problem**.


Here's an example where we're choosing between lipschitz models with different lipschitz constants $B$. And, for the sake of theory, also $\mu$. 

In [ ]:
Bs = 2^(-5:5)
models = Bs |> map(function(B) {
  function(X,Y) { prediction.function(lipreg(X,Y,B=B)) }
})
names(models) = sprintf('lip %1.2f', Bs)
models = c(models, list(mu=function(X,Y) mu))

In [ ]:
n=100
sigma = .2
mu = function(x) { cos(1-2*x) }
#mu = function(x) { sin(pi/x) }
#mu = function(x) { x*sin(pi/x)}

set.seed(1)
X = runif(2*n)
epsilon = sigma*rnorm(length(X))
Y = mu(X) + epsilon

predictions = models |> map(function(fit.model) { 
  muhat = fit.model(X, Y)
  data.frame(X=X, muhat=muhat(X)) 
}) |> list_rbind(names_to='model')

x = seq(0,1,length.out=1000)
ggplot() +
  geom_line(aes(x=x, y=mu(x)), linewidth=1.5, alpha=.5) + 
  geom_point(aes(x=X, y=Y), alpha=.1) + 
  geom_line(aes(x=X, y=muhat, color=model), data=predictions, alpha=.5, linewidth=.2)  


Before we go further, let's ask whether the results we've proven apply here.
$$ \text{Is} \quad \mathcal{M} = \{ \hat \mu^1 \ldots \hat \mu^K \} \quad \text{ a model? }$$



The answer is no, not in the sense that we've been discussing. It's a
set of *random curves* that depend on the noise $\varepsilon_i$.
As a result, the projection
$$
\left\langle \varepsilon, \frac{m - \mu^\star}{\lVert m - \mu^\star \rVert} \right\rangle
$$
isn't a mean-zero normal random variable. That breaks our argument.

In [ ]:
error.replications = 1:1000 |> map(function(rep) {
  X = runif(2*n)
  epsilon = sigma*rnorm(length(X))
  Y = mu(X) + epsilon
  muhat = lipreg(X,Y) |> prediction.function()
  data.frame(rep=rep, projection = mean(epsilon * (muhat(X) - mu(X)) / sqrt(mean((muhat(X) - mu(X))^2))))
}) |> list_rbind()

In [ ]:
ggplot() + 
  geom_histogram(aes(x=projection, y=after_stat(density)), 
                 bins=30, data=error.replications, fill='red', color='gray', alpha=.2)

This breaks our argument; you may recall from our last lecture
that random curves aren't and can't be analyzed like (nonrandom) curves.
But there is an easy fix. Sample splitting!

## Sample Splitting

By fitting our $K$ curves $\hat\mu^1 \ldots \hat \mu^K$ on a sample
independent
$(\tilde X_1, \tilde Y_1) \ldots (\tilde X_{\tilde n}, \tilde Y_{\tilde n})$
different from the one $(X_1,Y_1) \ldots (X_n,Y_n)$ we use for
selection, we repair our argument. Sure, the curves we'll get are random
in the sense that they'll change if we resampled all of our data, but
they won't depend on $Y_1 \ldots Y_n$. That's all we need.

Here's a plot of the projection's distribution with sample splitting (in blue) and without it (in red).

In [ ]:
split.replications = 1:1000 |> map(function(rep) {
  X = runif(2*n)
  epsilon = sigma*rnorm(length(X))
  Y = mu(X) + epsilon
  train = (1:(2*n)) > n 
  muhat = lipreg(X[train],Y[train]) |> prediction.function()
  data.frame(rep=rep, projection = mean(epsilon[!train] * (muhat(X[!train]) - mu(X[!train])) / sqrt(mean((muhat(X[!train]) - mu(X[!train]))^2))))
}) |> list_rbind()

In [ ]:

ggplot() + 
  geom_histogram(aes(x=projection, y=after_stat(density)), 
                 bins=30, data=error.replications, fill='red', color='gray', alpha=.2) +
  geom_histogram(aes(x=projection, y=after_stat(density)), 
                 bins=30, data=split.replications, fill='blue', color='gray', alpha=.2)

### Details

Formally we'd say we *condition on* the sample used to fit
$\hat\mu^1 \ldots \hat\mu^K$. One way to think about that is that, if
we're using randomly-generated fake data to check that our error bound
is in fact valid with the probability we claim, we should estimate that
probability by checking how often it's valid when we regenerate the
selection sample $(X_1,Y_1) \ldots (X_n,Y_n)$ but not the training
sample
$(\tilde X_1, \tilde Y_1) \ldots (\tilde X_{\tilde n}, \tilde Y_{\tilde n})$.
That said, if a bound is valid with some probability for any given
training sample, it's going to be valid with (at least) that probability
on average over training samples. That is, if
$\lVert \hat\mu - \mu \rVert_{L_2(P_n)} \le s$ with probability
$1-\delta$ *conditional on*
$(\tilde X_1, \tilde Y_1) \ldots (\tilde X_{\tilde n}, \tilde Y_{\tilde n})$,
it's also true that $\lVert \hat\mu - \mu \rVert_{L_2(P_n)} \le s$ with
(unconditional) probability $1-\delta$.

### Procedure

1.  Split the data in half.
    $$ \underset{\text{first half}}{(X_1, Y_1) \ldots (X_n, Y_n)} \qquad
    \underset{\text{second half}}{(X_{n+1}, Y_{n+1}) \ldots (X_{2n}, Y_{2n})}. $$

2.  Use the second half to fit our $K$ models and get our $K$ least
    squares estimators.
    $$ \hat\mu^k = \operatorname*{argmin}_{m \in \mathcal{M}^k} \frac{1}{n}\sum_{i=n+1}^{2n} \{ Y_i - m(X_i)\}^2 \quad \text{ for } \quad k = 1 \ldots K. $$
    These $K$ curves go in our finite model $\mathcal{M}$. As far as the
    first half of the data knows, the curves aren't random. There's no
    dependence on the outcomes $Y_i$ in the first half.

3.  Using the first half, get least squares estimator $\hat \mu$ based
    on this finite model.
    $$ \hat \mu = \operatorname*{argmin}_{m \in \mathcal{M}} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i)\}^2 \quad \text{ for } \quad \mathcal{M} = \{ \hat\mu^1 \ldots \hat\mu^K \}. $$
    This is close to the target $\mu$ like we've proven, satisfying
    $$ \lVert \hat \mu - \mu\rVert_{L_2(P_n)} \le 4\sigma\sqrt{\log(K)/n} \quad \text{ with probability } \quad 1-1/K, $$
    **if** the target $\mu$ is in this finite model $\mathcal{M}$. But
    it won't be. So what we can say is this.
    $$ \lVert\hat\mu - \mu\rVert_{L_2(Pn)} \le 4\sigma\sqrt{\frac{\log(K)}{n}} + 3\lVert \mu^\star -\mu \rVert_{L_2(P_n)} \ \text{ with probability } \ 1-1/K \ \textbf{ for all } \ \mu^\star \in \mathcal{M}. $$
    In this context, what we're saying is that the selected curve
    $\hat\mu$ can't be much further than $\mu$ than the closest of the
    curves $\hat\mu^1 \ldots \hat \mu^K$ that we're selecting from.

### An Exercise

::: exercise
It seems wasteful not to use all the data to estimate your curves
$\hat\mu_1 \ldots \hat\mu_K$. Even if our theory doesn't apply, does it
make sense to do model selection without sample splitting, i.e. fitting
the curves $\hat\mu_1 \ldots \hat\mu_K$ on the same set we're using to select from them? 
How bad could it really be?

**Hint.** What happens when you fit the bounded variation model
$\{m : \rho_{TV}(m) \le B\}$ for larger and larger variation budgets
$B$?
:::

# Understanding Model Selection

Let's get to the heart of what it means to select a model. There are
really two interpretations out there.

1.  The first focuses on whether we're selecting *the right model*, for
    some sense of what that is. If you ever see the phrase *model
    selection consistency*, that's what they're talking about.
2.  The second focuses on how much we're paying for selecting a model
    automatically rather than knowing which will be best ahead of time.
    If you see the phrase *regret*, that's often what they're talking
    about. It's an evocative name. What we want is to ensure that, even
    if we haven't made 'the right choice' or it doesn't even make sense
    to think of there being one, we're not all that unhappy with our
    selection.

We'll do some exercises to help us to think about what's promised by the
results we've proven.


::: exercise
Do our results tell us that $\hat\mu$ is always the *closest* curve to
$\mu$ in our finite model $\mathcal{M}$? Why or why not?
:::


Here's the next exercise. Or really, a pair of them. Choose based on
your answer to the last one.

::: exercise
**If you answered yes.**

Let's think about whether there's something special about having the
smallest squared-error loss. If you took $\hat\mu_2$ to be the curve in
$\mathcal{M}$ with the second-smallest squared error loss, would it be
the *second-closest* curve to $\mu$ in our model?
:::


::: solution
You should've answered no. This question was meant to encourage you to
rethink your last answer if it were a yes. The correct answer to the
last question was, after all, that there's nothing special about the
curve with the smallest squared-error loss either.
:::

::: exercise
**If you answered no.**

Are there non-trivial models $\mathcal{M}$ for which the least squares
estimator $\hat\mu$ *is* always the closest curve to $\mu$ ? If so, what
is it about them that makes that happen? And in a model selection
context, would you expect to see them when selecting from the
least-squares curves $\hat\mu^K \ldots \hat\mu^K$ within some sets of
models $\mathcal{M}^1 \ldots \mathcal{M}^K$ and not other sets?
:::

::: solution
Basically no, unless our curves are well-separated.
:::

# Understanding Model Ranking

Now let's get into what is, in a sense, an expanded version of our last
exercise. When we choose $\hat\mu$ to be the curve in this model with
the smallest squared error loss, we're in effect *ranking* our curves in
order of increasing loss and then taking the first. We'll look into that
ranking.

It'll be important to keep in mind what that loss is. Here's what we
know.
$$ 
\ell(m)-\ell(\mu^\star) = \lVert m-\mu^\star\rVert_{L_2(P_n)}^2  - 2\left\langle\varepsilon, m - \mu^\star\right\rangle_{L_2(P_n)} - \left\langle \mu^\star-\mu, m - \mu^\star \right\rangle_{L_2(P_n)}  
$$
To keep things simple, we'll suppose that by some miracle, $\mu$ is in
our model $\mathcal{M}$. So we can think of our loss like this.
$$   
\ell(m)-\ell(\mu) = \lVert m-\mu\rVert_{L_2(P_n)}^2  - 2\left\langle\varepsilon, m - \mu \right\rangle_{L_2(P_n)} 
$$
We'll break this down, making a table with three columns: one for the
whole thing and one each for the two terms in this decomposition. In
real data, we don't know $\mu$ and therefore can't calculate
$\ell(m)-\ell(\mu)$, but that doesn't matter: $\ell(\mu)$ is a constant,
so we get the same ranking whether we look at this incalculable
difference or on the calculable quantity $\ell(m)$.

Let's start by calculating a model
$\mathcal{M}=\{\hat\mu^1 \ldots \hat\mu^K\}$. We'll use bounded
variation regression for a range of $B$. And we'll make sure to include
$\mu$. Then we'll rank our curves and thing about the result.


::: exercise
Look at the table generated by the code below. I've printed versions
sorted both by loss-difference and by distance to $\mu$. To what extent
does the ranking in terms of loss difference correspond to the ranking
in terms of distance to $\mu$? Why or why not?

After you've answered that, break down your answer a little bit. Think
about a few things. 

1. Are the curves with the *lowest* loss differences
all among the closest to $\mu$? 
2. Do the curves that are *furthest*
from $\mu$ ever have the lowest loss differences? 
3. Which of these matters in the argument we use to prove our bound?
:::

::: exercise
**Optional.** Is there another way of ranking curves that might get at
what's going on in our proof a little better? If you can think of one,
add it to your table and sort on it. See what you can see.
:::


In [ ]:
Bs = 2^(-5:5)
models = Bs |> map(function(B) {
  function(X,Y) { prediction.function(lipreg(X,Y,B=B)) }
})
names(models) = sprintf('lip %1.2f', Bs)
models = c(models, list(mu=function(X,Y) mu))

In [ ]:
n=100
sigma = .5
mu = function(x) { cos(1-2*x) }
#mu = function(x) { sin(pi/x) }
#mu = function(x) { x*sin(pi/x)}

set.seed(1)
X = runif(2*n)
epsilon = sigma*rnorm(length(X))
Y = mu(X) + epsilon

select = 1:n
fit  = (n+1) : (2*n)
fitted.models = models |> map(function(fit.model) { 
  muhat = fit.model(X[fit], Y[fit])
})

In [ ]:
loss = function(m) { mean( (Y[1:n] - m(X[1:n]))^2 ) }
dist.sq = function(m) { mean( (m(X[1:n]) - mu(X[1:n]))^2 ) }
mean.zero = function(m) { -2*mean( epsilon[1:n] * (m(X[1:n]) - mu(X[1:n])) ) }

error.table = map(fitted.models, function(muhat) {
  data.frame(loss.dif =  1000*(loss(muhat) - loss(mu)),
             dist.sq  =  1000*dist.sq(muhat),
             mean.zero = 1000*mean.zero(muhat),
             scaled.loss.dif=1000*(loss(muhat)-loss(mu))/sqrt(dist.sq(muhat)),
             scaled.mean.zero = 1000*mean.zero(muhat))
}) |> list_rbind(names_to='model')

Here's the table sorted by loss difference, $\ell(m)-\ell(\mu)$. 

In [ ]:
error.table[order(error.table$loss.dif),]

And here's a corresponding plot. It shows the Lipschitz regression estimators $\hat\mu$, with varying budgets $B$, we looked at above.
But now I've plotted the curves with *postive loss difference*, i.e. the ones with bigger squared error loss than $\mu$'s, as faint dotted lines.
We're not selecting one of these. The remaining curves, i.e. the ones with squared error loss no larger than $\mu$'s, are plotted as 
solid lines. Our error bound actually applies to every one of these. The thickest of these lines is the one we actually select, i.e. 
the one with the smallest squared error loss.

In [ ]:
predictions = fitted.models |> map(function(muhat) { 
  data.frame(X=X, muhat=muhat(X)) 
}) |> list_rbind(names_to='model')

good.models = error.table$model[error.table$loss.dif <= 0]
best.model = error.table$model[error.table$loss.dif == min(error.table$loss.dif)]
x = seq(0,1,length.out=1000)
ggplot() +
  geom_line(aes(x=x, y=mu(x)), linewidth=1.5, alpha=.5) + 
  geom_point(aes(x=X, y=Y), alpha=.1) + 
  geom_line(aes(x=X, y=muhat, color=model), data=predictions[predictions$model == best.model,], alpha=.5, linewidth=1)  +
  geom_line(aes(x=X, y=muhat, color=model), data=predictions[predictions$model %in% good.models,], alpha=.5, linewidth=.5)  +
  geom_line(aes(x=X, y=muhat, color=model), data=predictions, alpha=.5, linewidth=.25, linetype='dotted')  

Here's a few more ways of sorting the table.

In [ ]:
error.table[order(error.table$dist.sq), ]

In [ ]:
error.table[order(error.table$scaled.loss.dif), ]

# Proving our more general bound

To prove an error bound that doesn't require $\mu$ to be in our model,
we don't have to do much more than substitute $\mu^\star$ for $\mu$
itself in the argument we used to prove the bound that did. 

We can think of that argument as having three parts.

1.  *The high-level part.* We work out an condition involving this loss
    difference that implies $\hat \mu$ satisfies our bound.
2.  *The arithmetic part.* We characterize the squared-error loss
    difference $\ell(m)-\ell(\mu^\star)$ between an arbitrary curve $m$
    and our approximation $\mu^\star$ to $\mu$.
3.  *The synthesis part.* We show, using our characterization of this
    difference, that this condition holds with high probability.

You can do the first two parts in either order, but you've got to do
both of them before you do the third. Let's work through them. 


## The high-level part.

We're going to be talking about a *neighborhood* of $\mu^\star$ our
model.
$$ 
\mathcal{M}_s = \left\{ m \in \mathcal{M} : \lVert m - \mu^\star \rVert_{L_2(P_n)} \le s \right\} 
$$
What we're doing here is finding a condition---the one in bold
below---that implies $\hat \mu$ is in this neighborhood. This
neighborhood, of course, is a set of curves that's close to $\mu^\star$.
To get an implied statement about distance to $\mu$, we use the triangle
inequality. 
$$ 
\lVert \hat \mu - \mu \rVert_{L_2(P_n)} 
\le \lVert \hat \mu - \mu^\star \rVert_{L_2(P_n)} + \lVert \mu^\star - \mu \rVert_{L_2(P_n)} 
\le s + \lVert \mu^\star - \mu \rVert_{L_2(P_n)} \quad \text{ if } \quad \hat\mu \in \mathcal{M}_s 
$$ 
For now, we'll leave the *radius* $s$ of this neighborhood
unspecified. In the third part, we'll make the best choice we can.
Here's our argument.

What we know is that $\hat \mu$ matches (i.e. beats or ties) every other
curve in the model in terms of squared error loss. That's what a
minimizer (argmin) does.
$$ 
\hat \mu = \operatorname*{argmin}_{m \in \mathcal{M}} \ell(m) \quad \iff \quad \ell(\hat\mu) \le \ell(m) \ \text{ for all } \ m \in \mathcal{M} 
$$
For any approximation $\mu^\star$ to $\mu$ that's in our model, that
means it matches $\mu^\star$.
$$ 
\ell(\hat\mu) \le \ell(m) \ \text{ for all } \ m \in \mathcal{M} \implies \ell(\hat\mu) \le \ell(\mu^\star). 
$$
And **if no curve in our neighborhood's complement matches**
$\mu^\star$, this means $\hat\mu$ isn't in that complement.
$$ 
\ell(\hat\mu) \le \ell(\mu^\star) \ \textbf{ and } \ \ell(m) > \ell(\mu^\star) \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s \implies \hat\mu \not \in \mathcal{M}\setminus \mathcal{M}_s 
$$
And because $\hat\mu$ *is* in the model, that means $\hat\mu$ is in
the neighborhood
$$  
\hat\mu \not \in \mathcal{M}\setminus \mathcal{M}_s \ \textbf{ and } \  \hat\mu \in \mathcal{M} \quad \iff \quad \hat \mu \in \mathcal{M}_s 
$$

## The arithmetic part.

Here's where we characterize the loss differences we've been talking
about. Here's what we're after.
$$ \ell(m)-\ell(\mu^\star) = \lVert m-\mu^\star\rVert_{L_2(P_n)}^2  - 2\left\langle\varepsilon, m - \mu^\star\right\rangle_{L_2(P_n)} - \left\langle \mu^\star-\mu, m - \mu^\star \right\rangle_{L_2(P_n)} $$
We've got three terms. From left to right, they're the squared sample
two-norm of the difference $m-\mu^\star$, a mean-zero normal term with
standard deviation proportional to this two-norm (not-squared), and a
last term that scales with both the difference $m-\mu^\star$ and the
approximation error $\mu^\star - \mu$. Here's a derivation. 
$$
\begin{aligned}
\ell(m) - \ell(\mu^\star) 
&= \frac{1}{n}\sum_{i=1}^n  \{Y_i - m(X_i)\}^2 - \frac{1}{n}\sum_{i=1}^n\{ Y_i - \mu^\star(X_i) \}^2 \\
&= \frac{1}{n}\sum_{i=1}^n  [ \{Y_i - \mu^\star(X_i)\} + \{\mu^\star(X_i) - m(X_i)\} ]^2 - \{ Y_i - \mu(X_i) \}^2 \\
&= \frac{1}{n}\sum_{i=1}^n  \{\mu^\star(X_i) - m(X_i)\}^2 - \frac{2}{n}\sum_{i=1}^n \{ Y_i - \mu^\star(X_i) \}\{m(X_i) - \mu^\star(X_i))\} \\
&= \frac{1}{n}\sum_{i=1}^n  \{\mu^\star(X_i) - m(X_i)\}^2 - \frac{2}{n}\sum_{i=1}^n \{Y_i - \mu(X_i)\} \{ m(X_i) - \mu^\star(X_i) \} \\
&- \frac{2}{n}\sum_{i=1}^n \{\mu(X_i) - \mu^\star(X_i) \}\{m(X_i) - \mu^\star(X_i) \}.
\end{aligned}
$$ 
Our characterization above is just this in vector notation.

Here's what we need to know about our terms. 
$$ 
\begin{aligned}
2\left\langle \varepsilon, m - \mu^\star\right\rangle_{L_2(P_n)} &\overset{(a)}{=} \frac{2 \sigma \lVert m-\mu^\star \rVert_{L_2(P_n)}}{\sqrt{n}}Z_m \quad \text{ where } \quad Z_m \sim N(0,1) \\
2\left\langle \mu^\star - \mu, m - \mu^\star\right\rangle_{L_2(P_n)} &\overset{(b)}{\le} 2\lVert \mu - \mu^\star \rVert_{L_2(P_n)} \lVert m-\mu^\star \rVert_{L_2(P_n)}
\end{aligned}
$$

a.  This is true because a weighted average of the mean-zero normal
    random variables ($\varepsilon_i$) is mean-zero normal. And we can
    write any mean-zero normal as its standard deviation times a
    standard normal. That's what we've done. We calculated the standard
    deviation to be what's in front of $Z_m$ last lecture.

b.  This is an application of the Cauchy-Schwarz inequality. We saw that
    on the Inner Product Spaces Homework.

## The synthesis part.

The high-level part promises us a bound if no curve in our
neighborhood's complement matches $\mu^\star$ in terms of squared error
loss.
$$ \lVert \hat \mu - \mu \rVert_{L_2(P_n)} \le s + \lVert \mu^\star - \mu \rVert_{L_2(P_n)} \quad \text{ if } \quad \ell(m) > \ell(\mu^\star) \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s $$
The bound we've claimed is this one with
$s=4\sigma\sqrt{\frac{\log(K)}{n}} + 2\lVert \mu^\star -\mu \rVert_{L_2(P_n)}$.
So what we need to show to prove it is that our premise, i.e. what's to
the right of the **if** above, is true with the claimed probability
$1-1/K$. Let's do it.

We need to show that with the claimed probability,
$$ 
\ell(m) - \ell(\mu^\star) > 0  \quad \text{ for all } \quad m \in \mathcal{M} \setminus \mathcal{M}_s. 
$$
Plugging in our expansion of this loss difference, here's what this
means.
$$ 
\lVert m-\mu^\star\rVert_{L_2(P_n)}^2 - \frac{2\sigma \lVert m-\mu^\star \rVert_{L_2(P_n)}}{\sqrt{n}} Z_m 
   - 2\langle \mu - \mu^\star, m-\mu^\star \rangle_{L_2(P_n)} > 0 \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s 
$$
And because this last term is at least
$-2\lVert \mu-\mu^\star\rVert_{L_2(P_n)} \lVert m-\mu^\star\rVert_{L_2(P_n)}$,
this is true if
$$ 
\lVert m-\mu^\star\rVert_{L_2(P_n)}^2- \frac{2\sigma \lVert m-\mu^\star\rVert_{L_2(P_n)}}{\sqrt{n}}Z_m - 2\lVert \mu - \mu^\star\rVert_{L_2(P_n)}\lVert m-\mu^\star\rVert_{L_2(P_n)} > 0 \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s 
$$
or equivalently, crossing out common factors of
$\lVert m-\mu^\star\rVert_{L_2(P_n)}$ and rearranging, if
$$ 
\lVert m-\mu^\star\rVert_{L_2(P_n)} > \frac{2\sigma \lVert m-\mu^\star\rVert_{L_2(P_n)}}{\sqrt{n}}Z_m + 2\lVert \mu - \mu^\star\rVert_{L_2(P_n)} \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s 
$$
This is true if the *minimum* of the left side (over $m$ in the
complement) is larger than the *maximum* of the right. 
$$ 
s = \min_{m \in \mathcal{M} \setminus \mathcal{M}_s} \lVert m-\mu^\star \rVert_{L_2(P_n)}  
> \frac{2\sigma}{\sqrt{n}}\max_{m \in \mathcal{M} \setminus \mathcal{M}_s} Z_m 
+ 2\lVert \mu - \mu^\star\rVert_{L_2(P_n)} \ \text{ for all } \ m \in \mathcal{M} \setminus \mathcal{M}_s. 
$$

To get our bound $s$, we plug in the standard upper bound
$2\sqrt{\log(K)}$ on the maximum of $K$ (or fewer) standard normals,
which holds with probability $1-1/K$. We're done.[^1]

[^1]: We proved that standard bound in the last lecture, for what it's
    worth.
